In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
huggingface_token = os.getenv("HF_TOKEN_read")

from huggingface_hub import login
login(token=huggingface_token)

In [2]:
from transformers import AutoModelForCausalLM
import torch

base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    torch_dtype=torch.bfloat16,
    device_map = {"": 0}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
fine_tuned_model_path = f"../../Ophtimus_LoRA/Ophtimus_3B_Instruct_checkpoint/checkpoint-1584"

In [4]:
# 16bit model + adapter
from peft import PeftConfig, PeftModel
base_and_adapter_model = PeftModel.from_pretrained(base_model, fine_tuned_model_path)

This can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64



In [5]:
base_and_adapter_model = base_and_adapter_model.merge_and_unload()

In [6]:
# 모델을 로컬 디렉토리에 저장
save_path = f"../../Ophtimus/Ophtimus-3B-Instruct-v1/1epoch"  # 저장할 경로를 지정

import os
os.makedirs(save_path, exist_ok=True)

base_and_adapter_model.save_pretrained(save_path)